<a href="https://colab.research.google.com/github/hosseinmiripy/rezume/blob/main/tradingsignal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install pandas numpy ccxt requests multiprocessing-on-dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 4.4 MB/s eta 0:00:00
  Created wheel for multiprocessing-on-dill: filename=multiprocessing_on_dill-3.5.0a4-py3-none-any.whl size=64017 sha256=e8b3ca667c6b9019e987e96578d93d3607140539e379d8f7f4ec9415f52fbb47
  Stored in directory: /root/.cache/pip/wheels/55/31/39/4c3d29d345c11e257f46f60d682bed191732f4d16584c34071
Successfully built multiprocessing-on-dill


In [15]:
import pandas as pd
import numpy as np
import time
import random
import requests
from multiprocessing import Pool
import ccxt
import warnings

# غیرفعال کردن هشدارها برای خوانایی بهتر خروجی
warnings.filterwarnings('ignore')

# تنظیم حداکثر ستون‌ها و ردیف‌های نمایش داده‌شده توسط پانداس
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

In [ ]:
import pandas as pd
import numpy as np
import time
import random
import requests
from multiprocessing import Pool
import ccxt
import warnings

warnings.filterwarnings('ignore')

# تنظیمات تلگرام
TELEGRAM_API_TOKEN = '7989027484:AAHtz4q396jSU2XmGMDOW27vjkmsDvN0i8w'
CHANNEL_ID = '@hosseinbtb2'


def send_message_to_telegram(message):
    """
    ارسال پیام به کانال تلگرام با بررسی کلیدواژه‌های مهم
    """
    keywords = [
        "فرصت خرید ایده‌آل",
        "سیگنال خرید قوی",
        "فرصت فروش ایده‌آل",
        "سیگنال فروش قوی",
        "تأیید قوی",
        "فروش قوی",
        "خرید قوی",
        "هشدار پامپ",
        "هشدار دامپ"
    ]

    if 'تایم‌فریم' in message and any(keyword in message for keyword in keywords):
        try:
            url = f'https://api.telegram.org/bot{TELEGRAM_API_TOKEN}/sendMessage'
            payload = {'chat_id': CHANNEL_ID, 'text': message, 'parse_mode': 'Markdown'}
            response = requests.post(url, data=payload)
            print(f"پیام ارسال شد: {message[:100]}...")
            return response
        except Exception as e:
            print(f"خطا در ارسال پیام به تلگرام: {e}")
            return None
    else:
        print("شرایط لازم برای ارسال پیام وجود ندارد. هیچ پیامی ارسال نمی‌شود.")
        return None


def fetch_data(symbol, timeframe, limit=1000, max_retries=5, delay=2):
    """
    دریافت داده‌های قیمت با مکانیزم تلاش مجدد بهبود یافته
    """
    exchange = ccxt.kucoin()
    retries = 0
    while retries < max_retries:
        try:
            # تنظیم پارامترهای محدودیت تعداد کندل‌ها
            if timeframe in ['1m', '5m', '15m']:
                # برای تایم‌فریم‌های کوتاه، داده‌های بیشتری نیاز داریم
                actual_limit = min(1500, limit)
            else:
                actual_limit = limit

            ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=actual_limit)

            if len(ohlcv) < 50:  # حداقل داده‌های مورد نیاز
                print(f"⚠️ داده‌های ناکافی برای {symbol} در تایم‌فریم {timeframe}. تعداد: {len(ohlcv)}")
                return None

            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)

            # محاسبه تغییرات قیمت
            df['price_change'] = df['close'].pct_change() * 100

            # محاسبه حجم نرمال شده
            df['volume_change'] = df['volume'].pct_change() * 100
            df['norm_volume'] = df['volume'] / df['volume'].rolling(window=20).mean()

            return df

        except ccxt.RequestTimeout:
            print(
                f"⚠️ خطای زمان پاسخگویی برای {symbol} در تایم‌فریم {timeframe}. تلاش مجدد... ({retries + 1}/{max_retries})")
            retries += 1
            time.sleep(delay + random.uniform(0, 1))
        except Exception as e:
            print(f"🚨 خطای غیرمنتظره برای {symbol}: {e}")
            break

    print(f"❌ تلاش‌ها برای دریافت داده‌های {symbol} در تایم‌فریم {timeframe} به پایان رسید.")
    return None


def calculate_rsi(df, period=14):
    """
    محاسبه شاخص قدرت نسبی (RSI) با مدیریت بهتر موارد خاص
    """
    df = df.copy()
    delta = df['close'].diff()

    # جداسازی تغییرات مثبت و منفی
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # محاسبه میانگین متحرک نمایی برای دقت بیشتر
    avg_gain = gain.ewm(com=period - 1, min_periods=period).mean()
    avg_loss = loss.ewm(com=period - 1, min_periods=period).mean()

    # جلوگیری از تقسیم بر صفر
    avg_loss = avg_loss.replace(0, 0.001)

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi


def calculate_rsi_divergence(df, window=14):
    """
    تشخیص واگرایی RSI با استفاده از روش پیشرفته
    """
    price = df['close']
    rsi = calculate_rsi(df, period=window)

    # یافتن قله‌ها و دره‌های محلی
    def find_local_peaks(series, window=5):
        peaks = pd.Series(0, index=series.index)
        troughs = pd.Series(0, index=series.index)

        for i in range(window, len(series) - window):
            if all(series.iloc[i] > series.iloc[i - j] for j in range(1, window + 1)) and \
                    all(series.iloc[i] > series.iloc[i + j] for j in range(1, window + 1)):
                peaks.iloc[i] = 1

            if all(series.iloc[i] < series.iloc[i - j] for j in range(1, window + 1)) and \
                    all(series.iloc[i] < series.iloc[i + j] for j in range(1, window + 1)):
                troughs.iloc[i] = 1

        return peaks, troughs

    price_peaks, price_troughs = find_local_peaks(price)
    rsi_peaks, rsi_troughs = find_local_peaks(rsi)

    # حداکثر دو قله/دره آخر را بررسی کن
    price_peak_indices = np.where(price_peaks == 1)[0][-2:] if sum(price_peaks) >= 2 else []
    price_trough_indices = np.where(price_troughs == 1)[0][-2:] if sum(price_troughs) >= 2 else []
    rsi_peak_indices = np.where(rsi_peaks == 1)[0][-2:] if sum(rsi_peaks) >= 2 else []
    rsi_trough_indices = np.where(rsi_troughs == 1)[0][-2:] if sum(rsi_troughs) >= 2 else []

    # شناسایی واگرایی منفی (قیمت بالاتر، RSI پایین‌تر)
    bearish_div = False
    if len(price_peak_indices) == 2 and len(rsi_peak_indices) == 2:
        if price.iloc[price_peak_indices[1]] > price.iloc[price_peak_indices[0]] and \
                rsi.iloc[rsi_peak_indices[1]] < rsi.iloc[rsi_peak_indices[0]]:
            bearish_div = True

    # شناسایی واگرایی مثبت (قیمت پایین‌تر، RSI بالاتر)
    bullish_div = False
    if len(price_trough_indices) == 2 and len(rsi_trough_indices) == 2:
        if price.iloc[price_trough_indices[1]] < price.iloc[price_trough_indices[0]] and \
                rsi.iloc[rsi_trough_indices[1]] > rsi.iloc[rsi_trough_indices[0]]:
            bullish_div = True

    return {
        'bullish_divergence': bullish_div,
        'bearish_divergence': bearish_div
    }


def calculate_macd(df, fast=12, slow=26, signal=9):
    """
    محاسبه شاخص MACD با بهبود دقت
    """
    df = df.copy()
    # استفاده از میانگین متحرک نمایی برای دقت بیشتر
    ema_fast = df['close'].ewm(span=fast, adjust=False).mean()
    ema_slow = df['close'].ewm(span=slow, adjust=False).mean()

    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    histogram = macd_line - signal_line

    # محاسبه تغییرات هیستوگرام برای شناسایی بهتر روند
    histogram_change = histogram.diff()

    return macd_line, signal_line, histogram, histogram_change


def calculate_stochastic(df, k_period=14, d_period=3, slowing=3):
    """
    محاسبه اسیلاتور استوکاستیک
    """
    df = df.copy()
    # محاسبه %K با دوره توقف برای کاهش نویز
    low_min = df['low'].rolling(window=k_period).min()
    high_max = df['high'].rolling(window=k_period).max()

    # از تقسیم بر صفر جلوگیری کنید
    denom = high_max - low_min
    denom = denom.replace(0, 0.001)

    k = 100 * ((df['close'] - low_min) / denom)
    k = k.rolling(window=slowing).mean()

    # محاسبه %D (میانگین متحرک ساده %K)
    d = k.rolling(window=d_period).mean()

    return k, d


def calculate_atr(df, period=14):
    """
    محاسبه میانگین دامنه واقعی (ATR) برای تخمین نوسان
    """
    df = df.copy()
    high_low = df['high'] - df['low']
    high_close = abs(df['high'] - df['close'].shift())
    low_close = abs(df['low'] - df['close'].shift())

    # بیشترین مقدار از سه محاسبه بالا را بگیرید
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = ranges.max(axis=1)

    # از میانگین متحرک نمایی برای واکنش بهتر به تغییرات استفاده کنید
    atr = true_range.ewm(span=period, min_periods=period).mean()

    return atr


def calculate_bollinger_bands(df, window=20, num_std=2):
    """
    محاسبه باندهای بولینگر با قابلیت تنظیم
    """
    df = df.copy()
    # استفاده از میانگین متحرک ساده
    sma = df['close'].rolling(window=window).mean()

    # محاسبه انحراف استاندارد
    std = df['close'].rolling(window=window).std()

    # محاسبه باندهای بالا و پایین
    upper_band = sma + (std * num_std)
    lower_band = sma - (std * num_std)

    # محاسبه عرض باند برای تشخیص فشرده‌سازی یا انبساط
    bandwidth = (upper_band - lower_band) / sma * 100

    # محاسبه موقعیت نسبی قیمت در باندها (0-100%)
    b_percent = (df['close'] - lower_band) / (upper_band - lower_band) * 100

    return upper_band, sma, lower_band, bandwidth, b_percent


def calculate_ichimoku(df):
    """
    محاسبه شاخص ایچیموکو برای تحلیل روند
    """
    df = df.copy()
    # محاسبه خطوط تنکانسن و کیجونسن
    high_9 = df['high'].rolling(window=9).max()
    low_9 = df['low'].rolling(window=9).min()
    df['tenkan_sen'] = (high_9 + low_9) / 2

    high_26 = df['high'].rolling(window=26).max()
    low_26 = df['low'].rolling(window=26).min()
    df['kijun_sen'] = (high_26 + low_26) / 2

    # محاسبه خط چیکو اسپن (قیمت با تأخیر 26 دوره)
    df['chikou_span'] = df['close'].shift(-26)

    # محاسبه سنکو اسپن A (میانگین تنکانسن و کیجونسن با پیش‌بینی 26 دوره)
    df['senkou_span_a'] = ((df['tenkan_sen'] + df['kijun_sen']) / 2).shift(26)

    # محاسبه سنکو اسپن B
    high_52 = df['high'].rolling(window=52).max()
    low_52 = df['low'].rolling(window=52).min()
    df['senkou_span_b'] = ((high_52 + low_52) / 2).shift(26)

    return df


def detect_volume_spikes(df, threshold=2.5):
    """
    شناسایی افزایش‌های ناگهانی در حجم معاملات
    """
    df = df.copy()
    # محاسبه میانگین متحرک حجم
    avg_volume = df['volume'].rolling(window=20).mean()

    # شناسایی حجم‌های غیرعادی
    df['volume_spike'] = df['volume'] / avg_volume

    # بررسی افزایش حجم ناگهانی در کندل‌های اخیر
    recent_spikes = df['volume_spike'].iloc[-5:] > threshold

    return recent_spikes.any(), df['volume_spike']


def detect_pump_dump_patterns(df, price_threshold=5.0, volume_threshold=3.0, window=48):
    """
    شناسایی الگوهای پامپ و دامپ با تشخیص تغییرات حجم و قیمت
    """
    df = df.copy()
    # محاسبه تغییرات قیمت و حجم
    df['price_pct_change'] = df['close'].pct_change() * 100
    df['cumulative_price_change'] = df['price_pct_change'].rolling(window=window).sum()

    # محاسبه نسبت حجم فعلی به میانگین
    df['volume_ratio'] = df['volume'] / df['volume'].rolling(window=20).mean()

    # تشخیص شرایط پامپ
    pump_conditions = (
            (df['price_pct_change'] > price_threshold) &  # افزایش ناگهانی قیمت
            (df['volume_ratio'] > volume_threshold)  # افزایش ناگهانی حجم
    )

    # تشخیص شرایط دامپ
    dump_conditions = (
            (df['price_pct_change'] < -price_threshold) &  # کاهش ناگهانی قیمت
            (df['volume_ratio'] > volume_threshold)  # افزایش ناگهانی حجم
    )

    # بررسی وقوع پامپ یا دامپ در کندل‌های اخیر
    recent_pump = pump_conditions.iloc[-3:].any()
    recent_dump = dump_conditions.iloc[-3:].any()

    # بررسی روند پیش از پامپ/دامپ احتمالی
    pre_pump_trend = df['close'].iloc[-10:-3].pct_change().mean() * 100

    # امتیاز پامپ یا دامپ (شدت)
    pump_score = 0
    dump_score = 0

    if recent_pump:
        # محاسبه شدت پامپ
        max_price_change = df['price_pct_change'].iloc[-3:].max()
        max_volume_ratio = df['volume_ratio'].iloc[-3:].max()
        pump_score = (max_price_change / price_threshold) * (max_volume_ratio / volume_threshold)

        # بررسی اگر قیمت قبلاً پایین بوده (احتمال پامپ بالاتر)
        if pre_pump_trend < -1.0:
            pump_score *= 1.5

    if recent_dump:
        # محاسبه شدت دامپ
        max_price_change = abs(df['price_pct_change'].iloc[-3:].min())
        max_volume_ratio = df['volume_ratio'].iloc[-3:].max()
        dump_score = (max_price_change / price_threshold) * (max_volume_ratio / volume_threshold)

        # بررسی اگر قیمت قبلاً بالا بوده (احتمال دامپ بالاتر)
        if pre_pump_trend > 1.0:
            dump_score *= 1.5

    # بررسی نشانه‌های زودهنگام پامپ یا دامپ
    early_pump_warning = False
    early_dump_warning = False

    # افزایش تدریجی حجم بدون تغییر قیمت (نشانه احتمالی تجمیع قبل از پامپ)
    if df['volume_ratio'].iloc[-5:].mean() > 1.5 and abs(df['price_pct_change'].iloc[-5:].mean()) < 0.5:
        if df['price_pct_change'].iloc[-5:].sum() > 0:
            early_pump_warning = True
        else:
            early_dump_warning = True

    return {
        'pump_detected': recent_pump,
        'dump_detected': recent_dump,
        'pump_score': pump_score,
        'dump_score': dump_score,
        'early_pump_warning': early_pump_warning,
        'early_dump_warning': early_dump_warning
    }


def analyze_market_structure(df):
    """
    تحلیل ساختار بازار با شناسایی سطوح کلیدی قیمت
    """
    df = df.copy()
    # شناسایی سویینگ های بالا و پایین
    window = 5  # اندازه پنجره برای شناسایی سویینگ

    swing_highs = []
    swing_lows = []

    for i in range(window, len(df) - window):
        # سویینگ بالا
        if all(df['high'].iloc[i] > df['high'].iloc[i - j] for j in range(1, window + 1)) and \
                all(df['high'].iloc[i] > df['high'].iloc[i + j] for j in range(1, window + 1)):
            swing_highs.append((df.index[i], df['high'].iloc[i]))

        # سویینگ پایین
        if all(df['low'].iloc[i] < df['low'].iloc[i - j] for j in range(1, window + 1)) and \
                all(df['low'].iloc[i] < df['low'].iloc[i + j] for j in range(1, window + 1)):
            swing_lows.append((df.index[i], df['low'].iloc[i]))

    # فقط سویینگ‌های اخیر را بررسی کنید
    recent_swings = 10
    swing_highs = swing_highs[-recent_swings:] if len(swing_highs) > recent_swings else swing_highs
    swing_lows = swing_lows[-recent_swings:] if len(swing_lows) > recent_swings else swing_lows

    # بررسی روند ساختاری
    current_price = df['close'].iloc[-1]

    # بررسی ساختار صعودی/نزولی
    bullish_structure = False
    bearish_structure = False

    if len(swing_highs) >= 2 and len(swing_lows) >= 2:
        latest_swing_high = swing_highs[-1][1]
        previous_swing_high = swing_highs[-2][1]
        latest_swing_low = swing_lows[-1][1]
        previous_swing_low = swing_lows[-2][1]

        # ساختار صعودی: سویینگ‌های بالا و پایین در حال افزایش
        bullish_structure = latest_swing_high > previous_swing_high and latest_swing_low > previous_swing_low

        # ساختار نزولی: سویینگ‌های بالا و پایین در حال کاهش
        bearish_structure = latest_swing_high < previous_swing_high and latest_swing_low < previous_swing_low

    # شناسایی سطوح کلیدی مقاومت و حمایت
    resistance_levels = [price for _, price in swing_highs if price > current_price]
    support_levels = [price for _, price in swing_lows if price < current_price]

    # یافتن نزدیک‌ترین سطوح
    nearest_resistance = min(resistance_levels) if resistance_levels else None
    nearest_support = max(support_levels) if support_levels else None

    return {
        'bullish_structure': bullish_structure,
        'bearish_structure': bearish_structure,
        'nearest_resistance': nearest_resistance,
        'nearest_support': nearest_support,
        'swing_highs': swing_highs,
        'swing_lows': swing_lows
    }


def fetch_market_sentiment():
    """
    دریافت احساسات بازار از منبع خارجی (شاخص ترس و طمع)
    """
    try:
        response = requests.get("https://api.alternative.me/fng/", timeout=10)
        data = response.json()
        sentiment = data['data'][0]['value_classification']
        value = int(data['data'][0]['value'])

        # ترجمه احساسات به فارسی
        sentiment_map = {
            "Extreme Fear": "ترس شدید",
            "Fear": "ترس",
            "Neutral": "خنثی",
            "Greed": "طمع",
            "Extreme Greed": "طمع شدید"
        }

        persian_sentiment = sentiment_map.get(sentiment, "نامشخص")
        return persian_sentiment, value
    except Exception as e:
        print(f"خطا در دریافت شاخص ترس و طمع: {e}")
        return "نامشخص", 50


def fetch_order_book(symbol, depth=20):  # تغییر depth به 20 (مقدار مجاز)
    """
    دریافت دفتر سفارش با عمق مناسب
    """
    try:
        exchange = ccxt.kucoin()
        order_book = exchange.fetch_order_book(symbol, limit=20)  # تغییر مقدار به 20
        return order_book
    except Exception as e:
        print(f"خطا در دریافت دفتر سفارش: {e}")
        return None

def analyze_order_book(order_book, current_price, depth=15):
    """
    تحلیل پیشرفته دفتر سفارش با شناسایی دیواره‌های خرید و فروش
    """
    if order_book is None:
        return "داده‌های دفتر سفارش در دسترس نیست", 0, 0, {}

    # تحلیل عمق دفتر سفارش
    bid_volume = sum([order[1] for order in order_book['bids'][:depth]])
    ask_volume = sum([order[1] for order in order_book['asks'][:depth]])

    # محاسبه نسبت خرید به فروش
    buy_sell_ratio = bid_volume / ask_volume if ask_volume > 0 else float('inf')

    # شناسایی دیوارهای خرید و فروش بزرگ (بیش از ۳ برابر میانگین)
    avg_bid_size = bid_volume / depth if depth > 0 else 0
    avg_ask_size = ask_volume / depth if depth > 0 else 0

    large_bid_orders = [order for order in order_book['bids'][:depth * 2] if order[1] > avg_bid_size * 3]
    large_ask_orders = [order for order in order_book['asks'][:depth * 2] if order[1] > avg_ask_size * 3]

    # محاسبه توزیع حجم در فواصل مختلف قیمت
    bid_distribution = {}
    ask_distribution = {}

    # تعریف محدوده‌های قیمتی (با فاصله 0.5% از قیمت فعلی)
    price_ranges = 5
    for i in range(price_ranges):
        lower_bid = current_price * (1 - (i + 1) * 0.005)
        upper_bid = current_price * (1 - i * 0.005)

        lower_ask = current_price * (1 + i * 0.005)
        upper_ask = current_price * (1 + (i + 1) * 0.005)

        # جمع حجم‌ها در هر محدوده
        bid_vol_in_range = sum([order[1] for order in order_book['bids']
                                if order[0] >= lower_bid and order[0] < upper_bid])

        ask_vol_in_range = sum([order[1] for order in order_book['asks']
                                if order[0] >= lower_ask and order[0] < upper_ask])

        bid_distribution[f"{lower_bid:.4f}-{upper_bid:.4f}"] = bid_vol_in_range
        ask_distribution[f"{lower_ask:.4f}-{upper_ask:.4f}"] = ask_vol_in_range

    # تفسیر نتایج
    if buy_sell_ratio > 2.0:
        book_signal = "فشار خرید قوی"
    elif buy_sell_ratio > 1.3:
        book_signal = "تمایل به خرید"
    elif buy_sell_ratio < 0.5:
        book_signal = "فشار فروش قوی"
    elif buy_sell_ratio < 0.77:
        book_signal = "تمایل به فروش"
    else:
        book_signal = "تعادل نسبی خرید و فروش"

    # اضافه کردن اطلاعات دیوارهای خرید و فروش
    walls_info = {}

    if large_bid_orders:
        # دیوار خرید نزدیک به قیمت فعلی
        nearest_bid_wall = min(large_bid_orders, key=lambda x: abs(x[0] - current_price))
        walls_info['nearest_bid_wall'] = nearest_bid_wall
        book_signal += f" - دیوار خرید قوی در {nearest_bid_wall[0]:.4f}"


    if large_ask_orders:
        # دیوار فروش نزدیک به قیمت فعلی
        nearest_ask_wall = min(large_ask_orders, key=lambda x: abs(x[0] - current_price))
        walls_info['nearest_ask_wall'] = nearest_ask_wall
        book_signal += f" - دیوار فروش قوی در {nearest_ask_wall[0]:.4f}"

    return book_signal, bid_volume, ask_volume, walls_info


def detect_whales_activity(df, order_book=None, volume_threshold=3.0):
    """
    شناسایی فعالیت نهنگ‌ها با بررسی حجم معاملات غیرعادی و دفتر سفارش
    """
    whale_signals = []

    # بررسی حجم‌های غیرعادی در کندل‌های اخیر
    avg_volume = df['volume'].rolling(window=20).mean()
    std_volume = df['volume'].rolling(window=20).std()

    # بررسی ۵ کندل آخر برای فعالیت غیرعادی
    for i in range(-5, 0):
        if df['volume'].iloc[i] > avg_volume.iloc[i] + (volume_threshold * std_volume.iloc[i]):
            # حجم بالای غیرعادی
            price_change = df['close'].iloc[i] / df['open'].iloc[i] - 1

            if abs(price_change) > 0.02:  # تغییر قیمت بیش از ۲ درصد
                direction = "صعودی" if price_change > 0 else "نزولی"
                intensity = "بسیار قوی" if df['volume'].iloc[i] > avg_volume.iloc[i] + (
                            4 * std_volume.iloc[i]) else "قوی"
                whale_signals.append(
                    f"فعالیت نهنگ {intensity} ({direction}) - تغییر قیمت {abs(price_change) * 100:.2f}% با حجم {df['volume'].iloc[i] / avg_volume.iloc[i]:.1f}X معمول")

    # بررسی دفتر سفارش برای سفارش‌های بزرگ
    if order_book:
        # محاسبه میانگین حجم سفارش
        avg_bid_size = sum([order[1] for order in order_book['bids'][:20]]) / 20 if len(order_book['bids']) >= 20 else 0
        avg_ask_size = sum([order[1] for order in order_book['asks'][:20]]) / 20 if len(order_book['asks']) >= 20 else 0

        # شناسایی سفارش‌های بسیار بزرگ (بیش از ۱۰ برابر میانگین)
        very_large_bids = [order for order in order_book['bids'][:20] if order[1] > avg_bid_size * 10]
        very_large_asks = [order for order in order_book['asks'][:20] if order[1] > avg_ask_size * 10]

        if very_large_bids:
            for order in very_large_bids[:2]:  # فقط ۲ مورد بزرگترین را گزارش کن
                whale_signals.append(f"سفارش خرید بسیار بزرگ: {order[1]:.2f} واحد در قیمت {order[0]:.4f}")

        if very_large_asks:
            for order in very_large_asks[:2]:  # فقط ۲ مورد بزرگترین را گزارش کن
                whale_signals.append(f"سفارش فروش بسیار بزرگ: {order[1]:.2f} واحد در قیمت {order[0]:.4f}")

    return whale_signals


def calculate_support_resistance(df, n_levels=3, price_tolerance=0.005):
    """
    محاسبه سطوح حمایت و مقاومت کلیدی با استفاده از تحلیل حجم و قیمت
    """
    df = df.copy()

    # شناسایی نقاط کلیدی قیمت با استفاده از رفتار قیمت و حجم
    price_levels = {}

    for i in range(20, len(df) - 1):
        # بررسی برگشت‌های قیمتی با حجم بالا
        if (df['high'].iloc[i] > df['high'].iloc[i - 1] and df['high'].iloc[i] > df['high'].iloc[i + 1]) or \
                (df['low'].iloc[i] < df['low'].iloc[i - 1] and df['low'].iloc[i] < df['low'].iloc[i + 1]):

            # برگشت قیمت همراه با حجم بالاتر از میانگین
            if df['volume'].iloc[i] > df['volume'].iloc[i - 10:i].mean():
                # سطح قیمت را به عنوان میانگین کندل ثبت کن
                price_level = (df['high'].iloc[i] + df['low'].iloc[i]) / 2

                # محدوده قیمت را به عنوان کلید استفاده کن (با تلرانس)
                price_range = round(price_level / price_tolerance) * price_tolerance

                if price_range in price_levels:
                    price_levels[price_range]['count'] += 1
                    price_levels[price_range]['volume'] += df['volume'].iloc[i]
                else:
                    price_levels[price_range] = {
                        'count': 1,
                        'volume': df['volume'].iloc[i],
                        'price': price_level
                    }

    # مرتب‌سازی بر اساس تعداد تکرار و حجم
    sorted_levels = sorted(price_levels.items(),
                           key=lambda x: (x[1]['count'], x[1]['volume']),
                           reverse=True)

    # انتخاب مهمترین سطوح
    important_levels = sorted_levels[:n_levels] if len(sorted_levels) > n_levels else sorted_levels

    # تقسیم‌بندی به سطوح حمایت و مقاومت نسبت به قیمت فعلی
    current_price = df['close'].iloc[-1]

    support_levels = []
    resistance_levels = []

    for _, level_info in important_levels:
        if level_info['price'] < current_price:
            support_levels.append((level_info['price'], level_info['count'], level_info['volume']))
        else:
            resistance_levels.append((level_info['price'], level_info['count'], level_info['volume']))

    # مرتب‌سازی بر اساس فاصله از قیمت فعلی
    support_levels = sorted(support_levels, key=lambda x: current_price - x[0])
    resistance_levels = sorted(resistance_levels, key=lambda x: x[0] - current_price)

    return support_levels, resistance_levels


def identify_key_levels(df, current_price):
    """
    شناسایی سطوح کلیدی قیمت نزدیک به قیمت فعلی
    """
    # محاسبه سطوح حمایت و مقاومت
    support_levels, resistance_levels = calculate_support_resistance(df, n_levels=5)

    # یافتن نزدیک‌ترین سطوح
    nearest_support = support_levels[0] if support_levels else None
    nearest_resistance = resistance_levels[0] if resistance_levels else None

    # بررسی فاصله قیمت تا سطوح
    proximity_to_level = False
    key_level_message = ""

    if nearest_support:
        support_price = nearest_support[0]
        distance_to_support = (current_price - support_price) / current_price

        if distance_to_support < 0.03:  # کمتر از ۳ درصد فاصله
            proximity_to_level = True
            support_strength = "بسیار قوی" if nearest_support[1] > 3 else "قوی"
            key_level_message = f"قیمت نزدیک به سطح حمایت {support_strength}: {support_price:.4f} (فاصله: {distance_to_support * 100:.1f}%)"

    if nearest_resistance:
        resistance_price = nearest_resistance[0]
        distance_to_resistance = (resistance_price - current_price) / current_price

        if distance_to_resistance < 0.03:  # کمتر از ۳ درصد فاصله
            proximity_to_level = True
            resistance_strength = "بسیار قوی" if nearest_resistance[1] > 3 else "قوی"
            # اگر قبلاً پیام سطح حمایت را اضافه کرده‌ایم، این را به آن اضافه کن
            if key_level_message:
                key_level_message += f"\nقیمت نزدیک به سطح مقاومت {resistance_strength}: {resistance_price:.4f} (فاصله: {distance_to_resistance * 100:.1f}%)"
            else:
                key_level_message = f"قیمت نزدیک به سطح مقاومت {resistance_strength}: {resistance_price:.4f} (فاصله: {distance_to_resistance * 100:.1f}%)"

    return proximity_to_level, key_level_message, nearest_support, nearest_resistance


def calculate_risk_reward(entry, stop_loss, take_profit):
    """
    محاسبه نسبت ریسک به ریوارد برای مدیریت معاملات
    """
    risk = abs(entry - stop_loss)
    reward = abs(take_profit - entry)

    if risk == 0:
        return 0

    return reward / risk


def analyze_multi_timeframe(symbol, current_timeframe):
    """
    تحلیل چند تایم‌فریمی برای تأیید سیگنال‌ها
    """
    # تعیین تایم‌فریم‌های مرتبط
    timeframe_hierarchy = {
        '5m': ['1m', '15m', '1h'],
        '15m': ['5m', '1h', '4h'],
        '30m': ['15m', '1h', '4h'],
        '1h': ['15m', '4h', '1d'],
        '4h': ['1h', '1d', '1w'],
        '1d': ['4h', '1w']
    }

    # اگر تایم‌فریم فعلی در سلسله‌مراتب تعریف نشده باشد
    if current_timeframe not in timeframe_hierarchy:
        return None

    related_timeframes = timeframe_hierarchy[current_timeframe]

    results = {
        'current': None,
        'higher': {},
        'lower': {}
    }

    # تحلیل تایم‌فریم فعلی
    df_current = fetch_data(symbol, current_timeframe)
    if df_current is not None and len(df_current) >= 50:
        results['current'] = analyze_single_timeframe(df_current, current_timeframe)

    # تحلیل تایم‌فریم‌های مرتبط
    for tf in related_timeframes:
        df = fetch_data(symbol, tf)
        if df is not None and len(df) >= 50:
            # مشخص کن تایم‌فریم بالاتر یا پایین‌تر است
            if timeframe_to_minutes(tf) > timeframe_to_minutes(current_timeframe):
                results['higher'][tf] = analyze_single_timeframe(df, tf)
            else:
                results['lower'][tf] = analyze_single_timeframe(df, tf)

    return results


def timeframe_to_minutes(timeframe):
    """
    تبدیل تایم‌فریم به دقیقه برای مقایسه
    """
    if 'm' in timeframe:
        return int(timeframe.replace('m', ''))
    elif 'h' in timeframe:
        return int(timeframe.replace('h', '')) * 60
    elif 'd' in timeframe:
        return int(timeframe.replace('d', '')) * 60 * 24
    elif 'w' in timeframe:
        return int(timeframe.replace('w', '')) * 60 * 24 * 7
    return 0


def analyze_single_timeframe(df, timeframe):
    """
    تحلیل جامع در یک تایم‌فریم با استفاده از اندیکاتورهای مختلف
    """
    # محاسبه شاخص‌های تکنیکال اصلی
    rsi = calculate_rsi(df)
    macd_line, signal_line, histogram, histogram_change = calculate_macd(df)
    upper_bb, middle_bb, lower_bb, bandwidth, b_percent = calculate_bollinger_bands(df)
    k, d = calculate_stochastic(df)
    atr = calculate_atr(df)

    # محاسبه میانگین‌های متحرک
    df['ema_9'] = df['close'].ewm(span=9, adjust=False).mean()
    df['ema_21'] = df['close'].ewm(span=21, adjust=False).mean()
    df['ema_50'] = df['close'].ewm(span=50, adjust=False).mean()
    df['ema_200'] = df['close'].ewm(span=200, adjust=False).mean()
    df['sma_50'] = df['close'].rolling(window=50).mean()
    df['sma_200'] = df['close'].rolling(window=200).mean()

    # تشخیص روند با ترکیب میانگین‌های متحرک مختلف
    current_price = df['close'].iloc[-1]

    # امتیازدهی به روند
    trend_score = 0

    # میانگین‌های متحرک نمایی
    if current_price > df['ema_9'].iloc[-1]:
        trend_score += 1
    if current_price > df['ema_21'].iloc[-1]:
        trend_score += 1
    if current_price > df['ema_50'].iloc[-1]:
        trend_score += 1.5
    if current_price > df['ema_200'].iloc[-1]:
        trend_score += 2

    # میانگین‌های متحرک ساده
    if current_price > df['sma_50'].iloc[-1]:
        trend_score += 1
    if current_price > df['sma_200'].iloc[-1]:
        trend_score += 1.5

    # روند میانگین‌های متحرک
    if df['ema_9'].iloc[-1] > df['ema_21'].iloc[-1]:
        trend_score += 1
    if df['ema_21'].iloc[-1] > df['ema_50'].iloc[-1]:
        trend_score += 1.5
    if df['ema_50'].iloc[-1] > df['ema_200'].iloc[-1]:
        trend_score += 2

    # تعیین وضعیت روند بر اساس امتیاز
    if trend_score >= 10:
        trend = "قویاً صعودی"
    elif trend_score >= 7:
        trend = "صعودی"
    elif trend_score >= 4:
        trend = "صعودی ضعیف"
    elif trend_score <= -10:
        trend = "قویاً نزولی"
    elif trend_score <= -7:
        trend = "نزولی"
    elif trend_score <= -4:
        trend = "نزولی ضعیف"
    else:
        trend = "خنثی"

    # بررسی RSI
    rsi_value = rsi.iloc[-1]
    if rsi_value > 75:
        rsi_status = "اشباع خرید شدید"
    elif rsi_value > 70:
        rsi_status = "اشباع خرید"
    elif rsi_value < 25:
        rsi_status = "اشباع فروش شدید"
    elif rsi_value < 30:
        rsi_status = "اشباع فروش"
    elif rsi_value > 60:
        rsi_status = "قوت نسبی"
    elif rsi_value < 40:
        rsi_status = "ضعف نسبی"
    else:
        rsi_status = "متعادل"

    # بررسی MACD
    if macd_line.iloc[-1] > signal_line.iloc[-1] and macd_line.iloc[-2] <= signal_line.iloc[-2]:
        macd_signal = "سیگنال خرید جدید"
    elif macd_line.iloc[-1] < signal_line.iloc[-1] and macd_line.iloc[-2] >= signal_line.iloc[-2]:
        macd_signal = "سیگنال فروش جدید"
    elif macd_line.iloc[-1] > signal_line.iloc[-1] and histogram_change.iloc[-1] > 0:
        macd_signal = "ادامه روند صعودی"
    elif macd_line.iloc[-1] < signal_line.iloc[-1] and histogram_change.iloc[-1] < 0:
        macd_signal = "ادامه روند نزولی"
    elif macd_line.iloc[-1] > signal_line.iloc[-1]:
        macd_signal = "صعودی"
    elif macd_line.iloc[-1] < signal_line.iloc[-1]:
        macd_signal = "نزولی"
    else:
        macd_signal = "خنثی"

    # بررسی باندهای بولینگر
    if current_price > upper_bb.iloc[-1]:
        bb_status = "بالاتر از باند بالایی"
    elif current_price < lower_bb.iloc[-1]:
        bb_status = "پایین‌تر از باند پایینی"
    elif b_percent.iloc[-1] > 80:
        bb_status = "نزدیک به باند بالایی"
    elif b_percent.iloc[-1] < 20:
        bb_status = "نزدیک به باند پایینی"
    else:
        bb_status = "بین باندها"

    # بررسی باریک شدن باندها (فشرده شدن)
    bb_squeeze = bandwidth.iloc[-1] < bandwidth.iloc[-26:].quantile(0.25)

    # بررسی استوکاستیک
    if k.iloc[-1] < 20 and d.iloc[-1] < 20:
        stoch_status = "اشباع فروش"
    elif k.iloc[-1] > 80 and d.iloc[-1] > 80:
        stoch_status = "اشباع خرید"
    elif k.iloc[-1] > d.iloc[-1] and k.iloc[-2] <= d.iloc[-2]:
        stoch_status = "سیگنال خرید"
    elif k.iloc[-1] < d.iloc[-1] and k.iloc[-2] >= d.iloc[-2]:
        stoch_status = "سیگنال فروش"
    elif k.iloc[-1] > d.iloc[-1]:
        stoch_status = "صعودی"
    elif k.iloc[-1] < d.iloc[-1]:
        stoch_status = "نزولی"
    else:
        stoch_status = "خنثی"

    # تحلیل مومنتوم
    momentum = df['close'].pct_change(periods=14).iloc[-1] * 100

    if momentum > 10:
        momentum_status = "مومنتوم صعودی قوی"
    elif momentum > 5:
        momentum_status = "مومنتوم صعودی"
    elif momentum < -10:
        momentum_status = "مومنتوم نزولی قوی"
    elif momentum < -5:
        momentum_status = "مومنتوم نزولی"
    else:
        momentum_status = "مومنتوم خنثی"

    # اندازه‌گیری نوسان با ATR
    atr_value = atr.iloc[-1]
    atr_percent = (atr_value / current_price) * 100

    if atr_percent > 5:
        volatility = "نوسان بسیار بالا"
    elif atr_percent > 3:
        volatility = "نوسان بالا"
    elif atr_percent < 1:
        volatility = "نوسان پایین"
    else:
        volatility = "نوسان متوسط"

    # شناسایی الگوهای کندل استیک (ساده)
    candle_pattern = identify_candle_patterns(df)

    return {
        'trend': trend,
        'rsi': rsi_value,
        'rsi_status': rsi_status,
        'macd_signal': macd_signal,
        'bollinger_status': bb_status,
        'bollinger_squeeze': bb_squeeze,
        'stochastic': stoch_status,
        'momentum': momentum_status,
        'volatility': volatility,
        'atr_percent': atr_percent,
        'candle_pattern': candle_pattern,
        'price': current_price,
        'timeframe': timeframe,
        'ema_50': df['ema_50'].iloc[-1],
        'ema_200': df['ema_200'].iloc[-1],
        'sma_50': df['sma_50'].iloc[-1],
        'sma_200': df['sma_200'].iloc[-1]
    }


def identify_candle_patterns(df):
    """
    شناسایی الگوهای ساده کندل استیک
    """
    if len(df) < 5:
        return "داده‌های ناکافی"

    patterns = []

    # دریافت کندل‌های اخیر
    df = df.copy()

    # محاسبه اندازه بدنه و سایه‌ها
    df['body_size'] = abs(df['close'] - df['open'])
    df['upper_shadow'] = df['high'] - df[['open', 'close']].max(axis=1)
    df['lower_shadow'] = df[['open', 'close']].min(axis=1) - df['low']
    df['candle_range'] = df['high'] - df['low']

    # بررسی الگوهای مختلف
    # چکش یا هنگینگ من (Hammer / Hanging Man)
    if (df['lower_shadow'].iloc[-1] > 2 * df['body_size'].iloc[-1] and
            df['upper_shadow'].iloc[-1] < 0.3 * df['body_size'].iloc[-1] and
            df['body_size'].iloc[-1] < 0.3 * df['candle_range'].iloc[-1]):
        if df['close'].iloc[-1] > df['open'].iloc[-1]:
            patterns.append("چکش صعودی")
        else:
            patterns.append("هنگینگ من")

    # ستاره شوتینگ (Shooting Star)
    if (df['upper_shadow'].iloc[-1] > 2 * df['body_size'].iloc[-1] and
            df['lower_shadow'].iloc[-1] < 0.3 * df['body_size'].iloc[-1] and
            df['body_size'].iloc[-1] < 0.3 * df['candle_range'].iloc[-1]):
        patterns.append("ستاره شوتینگ")

    # دوجی (Doji)
    if df['body_size'].iloc[-1] < 0.1 * df['candle_range'].iloc[-1]:
        patterns.append("دوجی")

    # انگالفینگ (Engulfing)
    if len(df) > 2:
        if (df['close'].iloc[-1] > df['open'].iloc[-1] and  # کندل سبز
                df['close'].iloc[-2] < df['open'].iloc[-2] and  # کندل قرمز قبلی
                df['close'].iloc[-1] > df['open'].iloc[-2] and
                df['open'].iloc[-1] < df['close'].iloc[-2]):
            patterns.append("انگالفینگ صعودی")

        if (df['close'].iloc[-1] < df['open'].iloc[-1] and  # کندل قرمز
                df['close'].iloc[-2] > df['open'].iloc[-2] and  # کندل سبز قبلی
                df['close'].iloc[-1] < df['open'].iloc[-2] and
                df['open'].iloc[-1] > df['close'].iloc[-2]):
            patterns.append("انگالفینگ نزولی")

    # کندل بزرگ نشان‌دهنده فشار خرید/فروش
    avg_body = df['body_size'].iloc[-10:-1].mean()
    if df['body_size'].iloc[-1] > 2 * avg_body:
        if df['close'].iloc[-1] > df['open'].iloc[-1]:
            patterns.append("کندل صعودی قوی")
        else:
            patterns.append("کندل نزولی قوی")

    # ترکیب نتایج
    if not patterns:
        return "الگوی خاصی شناسایی نشد"
    return " - ".join(patterns)


def determine_signal_strength(mtf_analysis, divergences, key_level_info, pump_dump_info, whale_signals,
                              order_book_signal):
    """
    تعیین قدرت سیگنال بر اساس ترکیب نتایج همه تحلیل‌ها
    """
    if mtf_analysis is None or 'current' not in mtf_analysis or mtf_analysis['current'] is None:
        return {
            'buy_score': 0,
            'sell_score': 0,
            'buy_confidence': "پایین",
            'sell_confidence': "پایین",
            'signal': "داده‌های ناکافی",
            'signal_type': None
        }

    # امتیازدهی برای سیگنال


def determine_signal_strength(mtf_analysis, divergences, key_level_info, pump_dump_info, whale_signals,
                              order_book_signal):
    """
    تعیین قدرت سیگنال بر اساس ترکیب نتایج همه تحلیل‌ها
    """
    if mtf_analysis is None or 'current' not in mtf_analysis or mtf_analysis['current'] is None:
        return {
            'buy_score': 0,
            'sell_score': 0,
            'buy_confidence': "پایین",
            'sell_confidence': "پایین",
            'signal': "داده‌های ناکافی",
            'signal_type': None
        }

    # امتیازدهی برای سیگنال‌های خرید و فروش
    buy_score = 0
    sell_score = 0
    max_score = 100  # امتیاز کل ممکن

    # 1. تحلیل تایم‌فریم فعلی (وزن: 35%)
    current = mtf_analysis['current']

    # بررسی روند کلی
    if "صعودی قویاً" in current['trend']:
        buy_score += 12
    elif "صعودی" in current['trend']:
        buy_score += 8
    elif "نزولی قویاً" in current['trend']:
        sell_score += 12
    elif "نزولی" in current['trend']:
        sell_score += 8

    # بررسی RSI
    if current['rsi_status'] == "اشباع فروش شدید":
        buy_score += 8
    elif current['rsi_status'] == "اشباع فروش":
        buy_score += 5
    elif current['rsi_status'] == "اشباع خرید شدید":
        sell_score += 8
    elif current['rsi_status'] == "اشباع خرید":
        sell_score += 5

    # بررسی MACD
    if "خرید جدید" in current['macd_signal']:
        buy_score += 7
    elif "فروش جدید" in current['macd_signal']:
        sell_score += 7
    elif "صعودی" in current['macd_signal']:
        buy_score += 4
    elif "نزولی" in current['macd_signal']:
        sell_score += 4

    # بررسی باندهای بولینگر
    if current['bollinger_status'] == "پایین‌تر از باند پایینی":
        buy_score += 6
    elif current['bollinger_status'] == "بالاتر از باند بالایی":
        sell_score += 6
    elif current['bollinger_status'] == "نزدیک به باند پایینی":
        buy_score += 3
    elif current['bollinger_status'] == "نزدیک به باند بالایی":
        sell_score += 3

    # بررسی استوکاستیک
    if current['stochastic'] == "اشباع فروش":
        buy_score += 5
    elif current['stochastic'] == "اشباع خرید":
        sell_score += 5
    elif current['stochastic'] == "سیگنال خرید":
        buy_score += 4
    elif current['stochastic'] == "سیگنال فروش":
        sell_score += 4

    # بررسی مومنتوم
    if "مومنتوم صعودی قوی" in current['momentum']:
        buy_score += 5
    elif "مومنتوم صعودی" in current['momentum']:
        buy_score += 3
    elif "مومنتوم نزولی قوی" in current['momentum']:
        sell_score += 5
    elif "مومنتوم نزولی" in current['momentum']:
        sell_score += 3

    # بررسی الگوهای کندل
    if any(pattern in current['candle_pattern'] for pattern in ["چکش صعودی", "انگالفینگ صعودی", "کندل صعودی قوی"]):
        buy_score += 5
    elif any(pattern in current['candle_pattern'] for pattern in
             ["ستاره شوتینگ", "هنگینگ من", "انگالفینگ نزولی", "کندل نزولی قوی"]):
        sell_score += 5

    # 2. بررسی تایم‌فریم‌های بالاتر (وزن: 25%)
    higher_bullish = 0
    higher_bearish = 0

    for tf, analysis in mtf_analysis['higher'].items():
        if "صعودی" in analysis['trend']:
            higher_bullish += 1
        elif "نزولی" in analysis['trend']:
            higher_bearish += 1

        # بررسی سیگنال‌های دیگر در تایم‌فریم بالاتر
        if analysis['macd_signal'] == "سیگنال خرید جدید":
            buy_score += 3
        elif analysis['macd_signal'] == "سیگنال فروش جدید":
            sell_score += 3

    # امتیاز کلی تایم‌فریم‌های بالاتر
    if len(mtf_analysis['higher']) > 0:
        if higher_bullish == len(mtf_analysis['higher']):  # همه تایم‌فریم‌های بالاتر صعودی
            buy_score += 15
        elif higher_bullish > higher_bearish:
            buy_score += 10
        elif higher_bearish == len(mtf_analysis['higher']):  # همه تایم‌فریم‌های بالاتر نزولی
            sell_score += 15
        elif higher_bearish > higher_bullish:
            sell_score += 10

    # 3. بررسی تایم‌فریم‌های پایین‌تر (وزن: 10%)
    lower_bullish = 0
    lower_bearish = 0

    for tf, analysis in mtf_analysis['lower'].items():
        if "صعودی" in analysis['trend']:
            lower_bullish += 1
        elif "نزولی" in analysis['trend']:
            lower_bearish += 1

    # امتیاز کلی تایم‌فریم‌های پایین‌تر
    if len(mtf_analysis['lower']) > 0:
        if lower_bullish == len(mtf_analysis['lower']):
            buy_score += 6
        elif lower_bullish > lower_bearish:
            buy_score += 4
        elif lower_bearish == len(mtf_analysis['lower']):
            sell_score += 6
        elif lower_bearish > lower_bullish:
            sell_score += 4

    # 4. بررسی واگرایی‌ها (وزن: 10%)
    if divergences.get('bullish_divergence', False):
        buy_score += 10
    if divergences.get('bearish_divergence', False):
        sell_score += 10

    # 5. بررسی نزدیکی به سطوح کلیدی (وزن: 10%)
    proximity_to_level, key_level_message, nearest_support, nearest_resistance = key_level_info

    if proximity_to_level:
        if "حمایت" in key_level_message:
            buy_score += 10
        elif "مقاومت" in key_level_message:
            sell_score += 10

    # 6. بررسی الگوهای پامپ و دامپ (وزن: 5%)
    if pump_dump_info['pump_detected']:
        sell_score += int(min(pump_dump_info['pump_score'] * 5, 5))
    if pump_dump_info['dump_detected']:
        buy_score += int(min(pump_dump_info['dump_score'] * 5, 5))

    # هشدار زودهنگام پامپ/دامپ
    if pump_dump_info['early_pump_warning']:
        buy_score += 3  # فرصت خرید قبل از پامپ
    if pump_dump_info['early_dump_warning']:
        sell_score += 3  # فرصت فروش قبل از دامپ

    # 7. بررسی فعالیت نهنگ‌ها (وزن: 5%)
    whale_buy_signals = 0
    whale_sell_signals = 0

    for signal in whale_signals:
        if "صعودی" in signal:
            whale_buy_signals += 1
        elif "نزولی" in signal:
            whale_sell_signals += 1

    if whale_buy_signals > whale_sell_signals:
        buy_score += 5
    elif whale_sell_signals > whale_buy_signals:
        sell_score += 5

    # 8. بررسی دفتر سفارش (وزن: 5%)
    if "فشار خرید قوی" in order_book_signal:
        buy_score += 5
    elif "تمایل به خرید" in order_book_signal:
        buy_score += 3
    elif "فشار فروش قوی" in order_book_signal:
        sell_score += 5
    elif "تمایل به فروش" in order_book_signal:
        sell_score += 3

    # نرمال‌سازی امتیازها به درصد
    buy_score = min(buy_score, max_score)
    sell_score = min(sell_score, max_score)

    # تعیین سطح اطمینان
    def get_confidence_level(score):
        if score >= 80:
            return "بسیار بالا"
        elif score >= 65:
            return "بالا"
        elif score >= 50:
            return "متوسط"
        elif score >= 35:
            return "پایین"
        else:
            return "بسیار پایین"

    buy_confidence = get_confidence_level(buy_score)
    sell_confidence = get_confidence_level(sell_score)

    # تعیین سیگنال نهایی
    signal = ""
    signal_type = None

    if buy_score >= 65 and buy_score > sell_score * 1.5:
        if buy_score >= 80:
            signal = "سیگنال خرید قوی"
        else:
            signal = "سیگنال خرید"
        signal_type = "buy"
    elif sell_score >= 65 and sell_score > buy_score * 1.5:
        if sell_score >= 80:
            signal = "سیگنال فروش قوی"
        else:
            signal = "سیگنال فروش"
        signal_type = "sell"
    else:
        signal = "سیگنال خنثی"
        signal_type = "neutral"

    # شرایط خاص برای سیگنال‌های ویژه با دقت بالا
    if buy_score >= 80 and proximity_to_level and "حمایت" in key_level_message and current['rsi_status'] in [
        "اشباع فروش", "اشباع فروش شدید"]:
        signal = "فرصت خرید ایده‌آل"
        signal_type = "strong_buy"

    if sell_score >= 80 and proximity_to_level and "مقاومت" in key_level_message and current['rsi_status'] in [
        "اشباع خرید", "اشباع خرید شدید"]:
        signal = "فرصت فروش ایده‌آل"
        signal_type = "strong_sell"

    # شرایط خاص برای هشدار پامپ و دامپ
    if pump_dump_info['pump_score'] > 2.5:
        signal = "هشدار پامپ"
        signal_type = "pump_alert"

    if pump_dump_info['dump_score'] > 2.5:
        signal = "هشدار دامپ"
        signal_type = "dump_alert"

    return {
        'buy_score': buy_score,
        'sell_score': sell_score,
        'buy_confidence': buy_confidence,
        'sell_confidence': sell_confidence,
        'signal': signal,
        'signal_type': signal_type
    }


def calculate_targets(df, signal_type, current_price, atr, nearest_support=None, nearest_resistance=None):
    """
    محاسبه نقاط ورود، حد ضرر و حد سود بر اساس نوع سیگنال
    """
    # محاسبه ATR به عنوان درصدی از قیمت
    atr_percent = atr / current_price

    targets = {
        'entry': current_price,
        'stop_loss': None,
        'take_profit_1': None,
        'take_profit_2': None,
        'take_profit_3': None,
        'risk_reward_ratio': None
    }

    if signal_type in ['buy', 'strong_buy']:
        # محاسبه حد ضرر برای سیگنال خرید
        if nearest_support and nearest_support[0] < current_price:
            # استفاده از سطح حمایت به عنوان پایه حد ضرر
            stop_loss = nearest_support[0] - (0.5 * atr)
        else:
            # استفاده از ATR برای محاسبه حد ضرر
            stop_loss = current_price - (2.5 * atr)

        # محاسبه اهداف قیمتی
        risk = current_price - stop_loss

        take_profit_1 = current_price + (1.5 * risk)  # نسبت ریسک/ریوارد 1.5
        take_profit_2 = current_price + (2.5 * risk)  # نسبت ریسک/ریوارد 2.5
        take_profit_3 = current_price + (4.0 * risk)  # نسبت ریسک/ریوارد 4.0

        # اگر مقاومت نزدیک وجود دارد، محدود کردن اهداف
        if nearest_resistance and nearest_resistance[0] > current_price:
            take_profit_1 = min(take_profit_1, nearest_resistance[0])

        targets['stop_loss'] = stop_loss
        targets['take_profit_1'] = take_profit_1
        targets['take_profit_2'] = take_profit_2
        targets['take_profit_3'] = take_profit_3

    elif signal_type in ['sell', 'strong_sell']:
        # محاسبه حد ضرر برای سیگنال فروش
        if nearest_resistance and nearest_resistance[0] > current_price:
            # استفاده از سطح مقاومت به عنوان پایه حد ضرر
            stop_loss = nearest_resistance[0] + (0.5 * atr)
        else:
            # استفاده از ATR برای محاسبه حد ضرر
            stop_loss = current_price + (2.5 * atr)

        # محاسبه اهداف قیمتی
        risk = stop_loss - current_price

        take_profit_1 = current_price - (1.5 * risk)  # نسبت ریسک/ریوارد 1.5
        take_profit_2 = current_price - (2.5 * risk)  # نسبت ریسک/ریوارد 2.5
        take_profit_3 = current_price - (4.0 * risk)  # نسبت ریسک/ریوارد 4.0

        # اگر حمایت نزدیک وجود دارد، محدود کردن اهداف
        if nearest_support and nearest_support[0] < current_price:
            take_profit_1 = max(take_profit_1, nearest_support[0])

        targets['stop_loss'] = stop_loss
        targets['take_profit_1'] = take_profit_1
        targets['take_profit_2'] = take_profit_2
        targets['take_profit_3'] = take_profit_3

    # محاسبه نسبت ریسک به ریوارد
    if targets['stop_loss'] is not None and targets['take_profit_1'] is not None:
        risk = abs(current_price - targets['stop_loss'])
        reward = abs(targets['take_profit_1'] - current_price)

        if risk != 0:
            targets['risk_reward_ratio'] = reward / risk
        else:
            targets['risk_reward_ratio'] = 0

    return targets


def generate_advanced_signal(symbol, timeframe, account_balance=1000):
    """
    تولید سیگنال پیشرفته با ترکیب همه تحلیل‌ها
    """
    # دریافت داده‌ها
    df = fetch_data(symbol, timeframe)
    if df is None or len(df) < 50:
        return "داده‌های ناکافی برای تحلیل", False

    # تحلیل چند تایم‌فریمی
    mtf_analysis = analyze_multi_timeframe(symbol, timeframe)

    # شناسایی واگرایی‌ها
    divergences = calculate_rsi_divergence(df)

    # دریافت دفتر سفارش
    order_book = fetch_order_book(symbol)

    # تحلیل دفتر سفارش
    current_price = df['close'].iloc[-1]
    order_book_analysis, bid_volume, ask_volume, walls_info = analyze_order_book(order_book,
                                                                                 current_price) if order_book else (
    "داده‌های دفتر سفارش در دسترس نیست", 0, 0, {})

    # شناسایی سطوح حمایت و مقاومت کلیدی
    key_level_info = identify_key_levels(df, current_price)

    # شناسایی الگوهای پامپ و دامپ
    pump_dump_info = detect_pump_dump_patterns(df)

    # شناسایی فعالیت نهنگ‌ها
    whale_signals = detect_whales_activity(df, order_book)

    # محاسبه ATR برای تعیین نوسان
    atr = calculate_atr(df).iloc[-1]

    # تعیین قدرت سیگنال
    signal_strength = determine_signal_strength(mtf_analysis, divergences, key_level_info, pump_dump_info,
                                                whale_signals, order_book_analysis)

    # اگر سیگنال خنثی است، سیگنال معتبری نیست
    if signal_strength['signal_type'] == 'neutral':
        return "سیگنال معتبری شناسایی نشد", False

    # محاسبه نقاط ورود، حد ضرر و حد سود
    targets = calculate_targets(df, signal_strength['signal_type'], current_price, atr, key_level_info[2],
                                key_level_info[3])

    # دریافت احساسات بازار
    market_sentiment, fear_index = fetch_market_sentiment()

    # تولید پیام سیگنال
    signal_message = f"🔍 {signal_strength['signal']} | {symbol}\n"
    signal_message += f"👁️ تایم‌فریم: {timeframe}\n"
    signal_message += f"💲 قیمت فعلی: {current_price:.6f}\n"
    signal_message += f"📊 اطمینان: {signal_strength['buy_confidence'] if 'buy' in signal_strength['signal_type'] else signal_strength['sell_confidence']}\n\n"

    # اضافه کردن جزئیات تحلیل
    if mtf_analysis and 'current' in mtf_analysis and mtf_analysis['current']:
        current = mtf_analysis['current']
        signal_message += f"📈 تحلیل تکنیکال:\n"
        signal_message += f"• روند: {current['trend']}\n"
        signal_message += f"• RSI: {current['rsi']:.1f} ({current['rsi_status']})\n"
        signal_message += f"• MACD: {current['macd_signal']}\n"
        signal_message += f"• بولینگر: {current['bollinger_status']}\n"

        if current['candle_pattern'] != "الگوی خاصی شناسایی نشد":
            signal_message += f"• الگوی کندل: {current['candle_pattern']}\n"

        signal_message += f"• مومنتوم: {current['momentum']}\n"
        signal_message += f"• نوسان: {current['volatility']} ({current['atr_percent']:.1f}%)\n"

    # اضافه کردن اطلاعات واگرایی
    if divergences['bullish_divergence'] or divergences['bearish_divergence']:
        signal_message += "\n🔄 واگرایی‌ها:\n"
        if divergences['bullish_divergence']:
            signal_message += "• واگرایی مثبت RSI (نشانه صعودی)\n"
        if divergences['bearish_divergence']:
            signal_message += "• واگرایی منفی RSI (نشانه نزولی)\n"

    # اضافه کردن اطلاعات سطوح کلیدی
    proximity_to_level, key_level_message, _, _ = key_level_info
    if proximity_to_level:
        signal_message += f"\n🔑 سطوح کلیدی:\n• {key_level_message}\n"

    # هشدار پامپ و دامپ
    if pump_dump_info['pump_detected'] or pump_dump_info['dump_detected'] or pump_dump_info['early_pump_warning'] or \
            pump_dump_info['early_dump_warning']:
        signal_message += "\n⚠️ هشدارها:\n"
        if pump_dump_info['pump_detected']:
            signal_message += f"• الگوی پامپ شناسایی شد (شدت: {pump_dump_info['pump_score']:.1f}/5)\n"
        if pump_dump_info['dump_detected']:
            signal_message += f"• الگوی دامپ شناسایی شد (شدت: {pump_dump_info['dump_score']:.1f}/5)\n"
        if pump_dump_info['early_pump_warning']:
            signal_message += "• نشانه‌های اولیه تجمیع قبل از پامپ احتمالی\n"
        if pump_dump_info['early_dump_warning']:
            signal_message += "• نشانه‌های اولیه توزیع قبل از دامپ احتمالی\n"

    # اطلاعات فعالیت نهنگ‌ها
    if whale_signals:
        signal_message += "\n🐋 فعالیت نهنگ‌ها:\n"
        for i, signal in enumerate(whale_signals[:2], 1):  # نمایش حداکثر ۲ مورد
            signal_message += f"• {signal}\n"

    # تحلیل دفتر سفارش
    if order_book:
        signal_message += f"\n📖 تحلیل دفتر سفارش:\n• {order_book_analysis}\n"
        signal_message += f"• نسبت خرید به فروش: {bid_volume / ask_volume:.2f}\n"

    # اضافه کردن اطلاعات احساسات بازار
    signal_message += f"\n🌍 احساسات بازار: {market_sentiment} (شاخص ترس و طمع: {fear_index})\n"

    # اضافه کردن اطلاعات مدیریت ریسک
    if targets['stop_loss'] is not None:
        signal_message += f"\n⚙️ مدیریت ریسک:\n"

        # محاسبه درصد تغییرات
        entry_price = current_price
        sl_percent = abs((targets['stop_loss'] - entry_price) / entry_price * 100)
        tp1_percent = abs((targets['take_profit_1'] - entry_price) / entry_price * 100)
        tp2_percent = abs((targets['take_profit_2'] - entry_price) / entry_price * 100)
        tp3_percent = abs((targets['take_profit_3'] - entry_price) / entry_price * 100)

        signal_message += f"• نقطه ورود: {entry_price:.6f}\n"
        signal_message += f"• حد ضرر: {targets['stop_loss']:.6f} ({sl_percent:.2f}%)\n"
        signal_message += f"• هدف اول: {targets['take_profit_1']:.6f} ({tp1_percent:.2f}%)\n"
        signal_message += f"• هدف دوم: {targets['take_profit_2']:.6f} ({tp2_percent:.2f}%)\n"
        signal_message += f"• هدف سوم: {targets['take_profit_3']:.6f} ({tp3_percent:.2f}%)\n"
        signal_message += f"• نسبت ریسک/ریوارد: {targets['risk_reward_ratio']:.2f}\n"

        # محاسبه اندازه پوزیشن پیشنهادی
        risk_percent = 2  # درصد ریسک از کل سرمایه
        position_size = (account_balance * risk_percent / 100) / sl_percent

        signal_message += f"• سرمایه در معرض ریسک: {risk_percent}% ({account_balance * risk_percent / 100:.2f} USDT)\n"
        signal_message += f"• اندازه پوزیشن پیشنهادی: {position_size:.4f} {symbol.split('/')[0]}\n"

    # افزودن توصیه نهایی
    signal_message += "\n📌 توصیه نهایی:\n"

    if signal_strength['signal_type'] == 'strong_buy':
        signal_message += "فرصت خرید بسیار مناسب با تأیید چندین شاخص. خرید در قیمت فعلی و رعایت حد ضرر پیشنهاد می‌شود."
    elif signal_strength['signal_type'] == 'buy':
        signal_message += "خرید با رعایت اصول مدیریت سرمایه و تنظیم حد ضرر پیشنهاد می‌شود."
    elif signal_strength['signal_type'] == 'strong_sell':
        signal_message += "فرصت فروش بسیار مناسب با تأیید چندین شاخص. فروش در قیمت فعلی و رعایت حد ضرر پیشنهاد می‌شود."
    elif signal_strength['signal_type'] == 'sell':
        signal_message += "فروش با رعایت اصول مدیریت سرمایه و تنظیم حد ضرر پیشنهاد می‌شود."
    elif signal_strength['signal_type'] == 'pump_alert':
        signal_message += "احتمال پامپ قیمت وجود دارد. معامله با احتیاط و حجم کم توصیه می‌شود. از تعیین حد ضرر مناسب اطمینان حاصل کنید."
    elif signal_strength['signal_type'] == 'dump_alert':
        signal_message += "احتمال دامپ قیمت وجود دارد. خروج از پوزیشن‌های خرید یا معامله با احتیاط توصیه می‌شود."

    # بررسی نسبت ریسک به ریوارد
    valid_signal = False
    if targets['risk_reward_ratio'] and targets['risk_reward_ratio'] >= 1.5:
        valid_signal = True
    elif signal_strength['signal_type'] in ['pump_alert', 'dump_alert']:
        valid_signal = True  # هشدارهای پامپ و دامپ همیشه معتبر هستند

    return signal_message, valid_signal


def process_analysis(args):
    """
    پردازش تحلیل برای یک جفت ارز و تایم‌فریم مشخص
    """
    symbol, timeframe, account_balance = args
    print(f"📊 در حال تحلیل {symbol} در تایم‌فریم {timeframe}...")

    try:
        signal, valid = generate_advanced_signal(symbol, timeframe, account_balance)

        if valid:
            print(f"✅ سیگنال معتبر برای {symbol} در تایم‌فریم {timeframe} شناسایی شد.")
            send_message_to_telegram(signal)
        else:
            print(f"❌ سیگنال معتبری برای {symbol} در تایم‌فریم {timeframe} شناسایی نشد.")

    except Exception as e:
        print(f"🚨 خطا در تحلیل {symbol} در تایم‌فریم {timeframe}: {e}")


def start_analysis(symbols, timeframes, account_balance=1000, sleep_time=300):
    """
    شروع تحلیل مستمر برای تمام جفت ارزها و تایم‌فریم‌ها
    """
    print(f"🚀 شروع تحلیل برای {len(symbols)} ارز در {len(timeframes)} تایم‌فریم...")

    while True:
        try:
            start_time = time.time()
            print(f"\n⏰ {time.strftime('%Y-%m-%d %H:%M:%S')} - شروع دور جدید تحلیل...")

            # لیست پارامترهای تحلیل
            tasks = [(symbol, timeframe, account_balance) for symbol in symbols for timeframe in timeframes]

            # اجرای موازی تحلیل‌ها
            with Pool(processes=8) as pool:  # استفاده از 8 پردازش همزمان
                pool.map(process_analysis, tasks)

            elapsed_time = time.time() - start_time
            print(f"⌛ زمان اجرای کامل: {elapsed_time:.2f} ثانیه")

            # انتظار تا دور بعدی
            remaining_time = max(sleep_time - elapsed_time, 10)
            print(f"💤 منتظر دور بعدی تحلیل... ({remaining_time:.0f} ثانیه)")
            time.sleep(remaining_time)

        except KeyboardInterrupt:
            print("\n🛑 برنامه با درخواست کاربر متوقف شد.")
            break
        except Exception as e:
            print(f"\n🚨 خطای اصلی: {e}")
            print("🔄 تلاش مجدد پس از 60 ثانیه...")
            time.sleep(60)


# تنظیمات ارزهای دیجیتال مورد تحلیل
symbols = [
    'BTC/USDT', 'ETH/USDT', 'XRP/USDT', 'SOL/USDT', 'ADA/USDT', 'AVAX/USDT',
    'DOT/USDT', 'LINK/USDT', 'UNI/USDT', 'AAVE/USDT', 'ATOM/USDT',
    'LTC/USDT', 'DOGE/USDT', 'SHIB/USDT', 'NEAR/USDT', 'TRX/USDT',
    'FIL/USDT', 'TON/USDT', 'XLM/USDT', 'ETC/USDT', 'ALGO/USDT', 'VET/USDT',
    'SAND/USDT', 'MANA/USDT', 'EGLD/USDT', 'AXS/USDT', 'ICP/USDT', 'XMR/USDT',
    '1INCH/USDT', 'ALPHA/USDT', 'ANKR/USDT', 'APE/USDT', 'AR/USDT', 'BAL/USDT',
    'BAND/USDT', 'BCH/USDT', 'CELO/USDT', 'CHR/USDT', 'CKB/USDT', 'COMP/USDT',
    'COTI/USDT', 'CRV/USDT', 'CTSI/USDT', 'CVX/USDT', 'DASH/USDT', 'DGB/USDT',
    'DYDX/USDT', 'EGLD/USDT', 'ENS/USDT', 'FLOW/USDT', 'GLM/USDT', 'GRT/USDT',
    'HBAR/USDT', 'HNT/USDT', 'ICX/USDT', 'IMX/USDT', 'JST/USDT', 'KAVA/USDT',
    'LDO/USDT', 'LPT/USDT', 'LRC/USDT', 'LSK/USDT', 'MEME/USDT', 'MKR/USDT',
    'MTL/USDT', 'NMR/USDT', 'OMG/USDT', 'OP/USDT', 'PEPE/USDT', 'PERP/USDT',
    'QNT/USDT', 'REN/USDT', 'ROSE/USDT', 'RSR/USDT', 'RUNE/USDT', 'RVN/USDT',
    'SNX/USDT', 'STX/USDT', 'SUI/USDT', 'SUSHI/USDT', 'SYS/USDT', 'TFUEL/USDT',
    'TRB/USDT', 'UMA/USDT', 'WAVES/USDT', 'WLD/USDT', 'XCH/USDT', 'YFI/USDT',
    'ZEC/USDT', 'ZEN/USDT', 'ZIL/USDT'
]

# تنظیم تایم‌فریم‌های مورد تحلیل
timeframes = ['15m', '1h', '4h']

# تنظیم مقدار سرمایه فرضی برای محاسبات مدیریت ریسک
account_balance = 100  # مقدار به USDT

# اجرای برنامه
if __name__ == '__main__':
    print("🔍 ربات تحلیل و سیگنال‌دهی ارزهای دیجیتال")
    print("=" * 50)
    print(f"🪙 تعداد ارزها: {len(symbols)}")
    print(f"⏱ تایم‌فریم‌ها: {', '.join(timeframes)}")
    print(f"💰 سرمایه در نظر گرفته شده: {account_balance} USDT")
    print(f"📡 کانال تلگرام: {CHANNEL_ID}")
    print("=" * 50)

    try:
        # شروع تحلیل و سیگنال‌دهی
        start_analysis(symbols, timeframes, account_balance)
    except KeyboardInterrupt:
        print("\n🛑 برنامه با درخواست کاربر متوقف شد.")
    except Exception as e:
        print(f"\n🚨 خطای اصلی برنامه: {e}")

🔍 ربات تحلیل و سیگنال‌دهی ارزهای دیجیتال
🪙 تعداد ارزها: 91
⏱ تایم‌فریم‌ها: 15m, 1h, 4h
💰 سرمایه در نظر گرفته شده: 100 USDT
📡 کانال تلگرام: @hosseinbtb2
🚀 شروع تحلیل برای 91 ارز در 3 تایم‌فریم...

⏰ 2025-03-08 23:16:17 - شروع دور جدید تحلیل...
📊 در حال تحلیل BTC/USDT در تایم‌فریم 15m...📊 در حال تحلیل DOT/USDT در تایم‌فریم 15m...📊 در حال تحلیل SOL/USDT در تایم‌فریم 15m...📊 در حال تحلیل AAVE/USDT در تایم‌فریم 15m...

📊 در حال تحلیل TRX/USDT در تایم‌فریم 15m...📊 در حال تحلیل XLM/USDT در تایم‌فریم 15m...

📊 در حال تحلیل DOGE/USDT در تایم‌فریم 15m...📊 در حال تحلیل VET/USDT در تایم‌فریم 15m...



❌ سیگنال معتبری برای DOT/USDT در تایم‌فریم 15m شناسایی نشد.
📊 در حال تحلیل DOT/USDT در تایم‌فریم 1h...
❌ سیگنال معتبری برای SOL/USDT در تایم‌فریم 15m شناسایی نشد.
📊 در حال تحلیل SOL/USDT در تایم‌فریم 1h...
❌ سیگنال معتبری برای VET/USDT در تایم‌فریم 15m شناسایی نشد.
📊 در حال تحلیل VET/USDT در تایم‌فریم 1h...
❌ سیگنال معتبری برای AAVE/USDT در تایم‌فریم 15m شناسایی نشد.
📊 در حال تحلیل AAVE/USDT در تایم‌